* Convert anndata to summarized experiment?
* Extract gene activity matrix from ArchR
* convert to scJoint input: https://github.com/sydneybiox/scJoint/blob/main/tutorial/sce_to_h5.ipynb

In [19]:
library(ArchR)
library(parallel)
library(zellkonverter)
library(SingleCellExperiment)
addArchRThreads(threads = 16) 
addArchRGenome('hg19')

Setting default number of Parallel threads to 16.

Setting default genome to Hg19.



# Read ATAC data 

In [2]:
data_path <- "/storage/groups/ml01/workspace/laura.martens/moretti_colab/archr_final/"

In [3]:
atac <- loadArchRProject(paste0(data_path, "Epicardiods"))

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .____

In [5]:
#get gene activity from archr project
gene_scores <- getMatrixFromProject(atac, useMatrix='GeneScoreMatrix')

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-4fb76dcea830-Date-2022-06-16_Time-18-30-07.log
If there is an issue, please report to github with logFile!

2022-06-16 18:33:36 : Organizing colData, 3.484 mins elapsed.

2022-06-16 18:33:37 : Organizing rowData, 3.49 mins elapsed.

2022-06-16 18:33:37 : Organizing rowRanges, 3.493 mins elapsed.

2022-06-16 18:33:37 : Organizing Assays (1 of 1), 3.494 mins elapsed.

2022-06-16 18:34:48 : Constructing SummarizedExperiment, 4.676 mins elapsed.

2022-06-16 18:34:50 : Finished Matrix Creation, 4.705 mins elapsed.



In [11]:
gene_scores <- as(gene_scores, "SingleCellExperiment")

In [107]:
counts(gene_scores) <- assays(gene_scores)$GeneScoreMatrix

In [17]:
rownames(gene_scores) <- rowData(gene_scores)$name

In [109]:
#create logcounts
libsizes <- colSums(counts(gene_scores))
size.factors <- libsizes/mean(libsizes)

In [110]:
logcounts(gene_scores) <- log2(t(t(counts(gene_scores))/size.factors) + 1)

In [111]:
gene_scores

class: SingleCellExperiment 
dim: 23127 39738 
metadata(0):
assays(3): GeneScoreMatrix counts logcounts
rownames(23127): DDX11L1 WASH7P ... CLIC2 TMLHE
rowData names(6): seqnames start ... name idx
colnames(39738): MUC26649_1234#ACTCGGGAGTTCCCGG-1
  MUC26649_1234#GATCGTAGTGATCAGG-1 ... MUC26653_1234#ATTACTCAGTCGAAAT-1
  MUC26653_1234#TAAACCGGTAGAAAGG-1
colData names(16): BlacklistRatio DoubletEnrichment ... TSSEnrichment
  projColData[, colnames(projColData) %ni% colnames(colData)]
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [112]:
8

[1] 8

# Load RNA

In [39]:
library(data.table)

In [40]:
data_path <- "/storage/groups/ml01/workspace/laura.martens/moretti_colab/transfer_data/"

In [36]:
#Convert(paste0(data_path, "Day2_15_RNAseq.h5ad"), dest = "h5seurat", overwrite = TRUE)

In [41]:
counts <- fread(paste0(data_path, "counts.csv"))

Warning message in fread(paste0(data_path, "counts.csv")):
"Detected 24354 column names but the data has 24355 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file."


In [49]:
mat <- as.matrix(counts[,3:ncol(counts)])

In [53]:
rownames(mat) <- counts$V1

In [71]:
mat <- t(mat)

In [82]:
coldata <- data.frame(clusters=as.factor(counts$clusters), row.names=colnames(mat))

In [83]:
rna <- SingleCellExperiment(assays = list(counts = mat), colData=coldata)

In [84]:
rna

class: SingleCellExperiment 
dim: 24353 35499 
metadata(0):
assays(1): counts
rownames(24353): RP11-34P13.7 AL627309.1 ... RP1-5O6.5 AP000695.1
rowData names(0):
colnames(35499): AAACCCAAGAGCAAGA-1_5 AAACCCAAGCCGCACT-1_7 ...
  TTTGTTGTCTCTCTAA-1_6 TTTGTTGTCTTAGGAC-1_1
colData names(1): clusters
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [ ]:
#create logcounts
libsizes <- colSums(mat)
size.factors <- libsizes/mean(libsizes)

logcounts(rna) <- log2(t(t(mat)/size.factors) + 1)

In [85]:
table(droplevels(rna$clusters))


   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15 
3460 2896 2644 2520 2208 2020 1930 1889 1486 1346 1130 1010  977  922  903  855 
  16   17   18   19   20   21   22   23   24   25   26   27   28   29   30 
 826  731  689  671  606  560  546  540  477  393  362  310  286  176  130 

# Save h5 data

In [113]:
# Only keep common genes between two dataset
common_genes <- intersect(rownames(rna),
                          rownames(gene_scores))
length(common_genes)

[1] 16949

In [137]:
# Extract the logcounts data from sce object
exprs_atac <- logcounts(gene_scores)[rownames(gene_scores) %in% common_genes]

In [138]:
exprs_atac

[1] 0.92143456 0.29042372 1.11769302 0.36176753 0.85837725 1.15704165
    [7] 0.68167273 0.38625826 0.96864193 0.13881413 0.61917691 0.00000000
   [13] 0.79742649 0.30801059 0.87420508 1.05796812 0.17504506 0.21163474
   [19] 0.62105419 0.80487801 0.31266444 0.66539139 0.23020246 0.36176753
   [25] 0.37517762 0.00000000 0.32192735 0.63319736 0.45943060 0.27500640
   [31] 0.38736166 0.00000000 0.32308104 0.51298421 1.21163313 0.23266020
   [37] 0.23143185 0.00000000 0.29513456 0.29160287 0.57434252 1.63551970
   [43] 0.00000000 1.12631010 1.01077795 0.39725444 0.53006959 0.30217209
   [49] 0.46152753 0.70663961 0.72071477 0.20914089 0.77483820 0.39066680
   [55] 0.45733061 0.43081953 1.44837002 1.01149375 0.77568114 0.65535046
   [61] 0.47612137 0.76553320 0.38294300 0.79659617 0.56462084 0.44995648
   [67] 0.42653218 0.71369435 0.62947187 0.75189062 0.40925422 0.94335697
   [73] 0.23511376 0.58496125 0.19156219 0.19408658 0.90457869 0.35501545
   [79] 0.40925422 0.76128374 0.84478569 0.35049644 0.73898245 0.37851076
   [85] 0.56266862 0.59931652 0.94785134 0.83187560 0.57240342 0.58976223
   [91] 2.08474176 0.41359315 0.35501545 0.23388750 0.23020246 0.76977019
   [97] 0.95531099 0.56071375 1.69554588 0.91303124 1.15898118 0.47196674
  [103] 0.35726965 0.53505643 0.90996332 0.44041974 0.34255395 1.00575739
  [109] 0.33113115 0.24488647 0.37406486 0.14012388 0.17759850 0.48851393
  [115] 0.52305892 0.46988494 0.51702254 0.82211665 0.78408357 1.31324361
  [121] 0.43188938 0.38183621 0.53505643 1.13553283 0.00000000 0.48645588
  [127] 0.34141574 0.77061610 1.04544104 0.59167816 0.22032942 0.62667143
  [133] 0.29630987 0.55679605 0.65076320 0.55973532 0.24853424 0.34936447
  [139] 0.77146151 0.73984660 0.49569407 0.40707985 0.89530089 0.82130041
  [145] 0.49262125 0.84558875 0.16864162 0.16735752 0.19282494 0.00000000
  [151] 0.03984016 0.27381360 0.89607632 1.97232225 0.78910164 0.56754423
  [157] 0.31033940 0.40272126 0.00000000 0.00000000 0.76807689 0.75445845
  [163] 0.75872804 0.22156726 0.60691466 0.27976776 0.27500640 0.25459344
  [169] 0.64061959 0.04544285 0.82211665 0.46884290 0.80239844 0.43295844
  [175] 1.16928158 1.50283734 0.32653659 0.15185844 0.64246919 0.40599144
  [181] 0.00000000 0.00000000 0.00000000 0.25096098 0.00000000 0.27976776
  [187] 0.39615859 0.17632234 0.00000000 0.25096098 0.90380782 0.56559596
  [193] 0.10164982 1.21412268 0.60786162 1.06488090 0.15185844 0.63040415
  [199] 0.40707985 0.39944366 0.43722676 0.00000000 0.00000000 0.19912216
  [205] 1.20351229 0.50182017 0.87656435 0.88986116 0.58303638 0.59836393
  [211] 0.00000000 0.30801059 0.23388750 0.42330829 0.75872804 0.54299970
  [217] 0.00000000 0.41575773 0.52305892 0.12432783 0.81311258 0.46780012
  [223] 0.00000000 0.43402670 0.84959734 1.12167653 0.52506545 0.71720883
  [229] 0.74845972 0.61447298 0.33571113 0.55385077 0.24122946 0.00000000
  [235] 0.00000000 0.08406405 0.92219608 0.66539139 0.12432783 0.72595775
  [241] 0.24366850 0.31150239 0.15055931 0.62105419 0.98986376 0.52907015
  [247] 0.13093055 0.64431643 1.33742340 0.24731935 0.57434252 0.00000000
  [253] 0.00000000 0.96569012 0.57821293 0.00000000 0.14273982 0.00000000
  [259] 0.19786491 0.00000000 0.64061959 0.52807002 0.31614501 0.24853424
  [265] 0.12961242 0.14665486 0.13224747 0.56462084 0.62853900 1.14404432
  [271] 1.04753646 0.12961242 0.31150239 0.24122946 0.65168182 1.34596176
  [277] 0.20288734 0.27022927 0.00000000 0.57724630 0.70398516 0.51803036
  [283] 0.24000839 0.43829187 0.91685700 1.17887187 1.34368980 0.53106834
  [289] 0.39176683 0.43402670 0.31614501 0.39944366 0.29395829 0.25096098
  [295] 0.86947494 0.43509418 1.20664110 0.29513456 1.28747008 0.92371791
  [301] 0.89452504 0.63412723 0.13881413 0.22280403 0.00000000 0.00000000
  [307] 0.66174822 0.63412723 0.44148249 0.19660657 0.32077273 0.55777647
  [313] 0.00000000 0.00000000 0.84879652 0.98476976 1.39176540 0.31498575
  [319] 0.73811779 0.50792035 0.42115501 0.17248709 0.15574885 0.28569745
  [325] 

In [124]:
exprs_rna <- logcounts(rna)[rownames(rna) %in% common_genes]

In [126]:
source("/storage/groups/ml01/code/laura.martens/github_repos/scJoint/data_to_h5.R")

Warning message:
"package 'HDF5Array' was built under R version 4.1.3"
Loading required package: DelayedArray


Attaching package: 'DelayedArray'


The following objects are masked from 'package:base':

    aperm, apply, rowsum, scale, sweep



Attaching package: 'HDF5Array'


The following object is masked from 'package:rhdf5':

    h5ls




In [128]:
write_h5_scJoint(exprs_list = list(rna = exprs_rna,
                                   atac = exprs_atac), 
                 h5file_list = c("/storage/groups/ml01/workspace/laura.martens/moretti_colab/scjoint/exprs_10xPBMC_rna.h5", 
                                 "/storage/groups/ml01/workspace/laura.martens/moretti_colab/scjoint/exprs_10xPBMC_atac.h5"))

In [140]:
write_h5_scJoint(exprs_list = list(atac = exprs_atac), 
                 h5file_list = c("/storage/groups/ml01/workspace/laura.martens/moretti_colab/scjoint/exprs_10xPBMC_atac.h5"))

ERROR: Error in UseMethod("h5writeDataset"): no applicable method for 'h5writeDataset' applied to an object of class "NULL"


In [134]:
write_csv_scJoint(cellType_list =  list(rna = rna$clusters),
                  csv_list = c("/storage/groups/ml01/workspace/laura.martens/moretti_colab/scjoint/cellType_10xPBMC_rna.csv"))